In [ ]:
import pandas as pd
from urllib.parse import quote

In [ ]:
window_micros = 200 * 1e6

startTs = 1705169091376222
endTs = startTs + window_micros

session = "[2023-12-09T04:14:00.672Z::854]"


def download_data(device: str):
    url = f"https://csiwiki.me.columbia.edu/rocketsdata2/export/0/{quote(session)}/{device}/records?startTs={startTs}&endTs={endTs}"
    return pd.read_csv(url)


mpu = download_data("MPU")
dht = download_data("DHT")

print("\nMPU:")
print(mpu.head())

print("\nDHT:")
print(dht.head())